In [ ]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier

from nltk.stem import WordNetLemmatizer
import nltk

# Loading dataset
data = pd.read_parquet('val-00000-of-00001-66ce8665444026dc.parquet')
data = data.dropna()

# Preprocessing	
# Extracting title and abstract from the dataset
X = data['title'] + ' ' + data['abstract']

nltk.download('wordnet')
# Lemmatization function
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(w) for w in text.split()])

X = X.apply(lemmatize_text)

# Extracting labels from the dataset (target)
Y_list = data['verified_uat_labels']

# Multi-label binarization of the target
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(Y_list)

# Parameters for the vectorizer and classifier
sgd_params = dict(alpha=1e-5, penalty="l2", loss="log_loss", n_jobs=-1, verbose=1)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

# Pipeline for text classification
pipeline = Pipeline(
    [   
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        ("clf", OneVsRestClassifier(SGDClassifier(**sgd_params))),
    ]
)

# Evaluation function
def eval_and_print_metrics(clf, X_train, y_train, X_test, y_test):
    global y_pred
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    # Calculate micro-averaged F1 score for multi-label classification
    print("Micro-averaged F1 score on test set: %0.3f" % f1_score(y_test, y_pred, average="micro"))
    print("-" * 10)

# Main script
if __name__ == "__main__":
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Supervised SGDClassifier with multi-label classification:")
    eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Quent\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Supervised SGDClassifier with multi-label classification:
-- Epoch 1
Norm: 32.73, NNZs: 19588, Bias: -2.004003, T: 2416, Avg. loss: 0.015290
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 30.31, NNZs: 19588, Bias: -2.457932, T: 4832, Avg. loss: 0.001864
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 29.25, NNZs: 19588, Bias: -2.671941, T: 7248, Avg. loss: 0.001980
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 28.16, NNZs: 19588, Bias: -2.853444, T: 9664, Avg. loss: 0.001734
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 26.95, NNZs: 19588, Bias: -3.035655, T: 12080, Avg. loss: 0.001502
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 26.33, NNZs: 19588, Bias: -3.153986, T: 14496, Avg. loss: 0.001640
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 25.66, NNZs: 19588, Bias: -3.272861, T: 16912, Avg. loss: 0.001525
Total training time: 0.01 seconds.
Convergence after 7 epochs took 0.01 seconds
-- Epoch 1
Norm: 36.82, NNZs: 19588, Bias: -2.054616, T: 2416, Avg. 

In [5]:
print(np.mean(y_pred == y_test))

# print the accuracy, precision, recall and f1_score
print("Accuracy: ", accuracy_score(y_pred,y_test))
print("Precision: ", precision_score(y_pred, y_test, average='micro'))
print("Recall: ", recall_score(y_pred, y_test, average='micro'))
print("F1 Score: ", f1_score(y_pred, y_test, average='micro'))


0.9949199950639628
Accuracy:  0.011589403973509934
Precision:  0.08326937835763622
Recall:  0.7281879194630873
F1 Score:  0.1494490358126722


In [6]:
# Doing my own accuracy calculation
# Search for each word in the predicted labels in the Y_test labels and calculate the percentage of words found
percentage = 0
for i in range(len(y_pred)):
    print(y_pred[i])
    print(y_test[i])
    if y_pred[i].tolist() == y_test[i].tolist():
        percentage += 1

percentage = percentage / len(y_pred)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 